In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

# Note name mapping (excluding octaves)
NOTE_NAMES = ['C', 'C#', 'D', 'D#', 'E', 'F',
              'F#', 'G', 'G#', 'A', 'A#', 'B']

# List of chord types
CHORD_TYPES = ['maj', 'min', 'dom', 'dim', 'aug']

# Assign colors to chord types (normalized to the 0-1 range for RGB)
CHORD_TYPE_COLORS = {
    'maj': (14/255, 40/255, 65/255),      # RGB (14, 40, 65)
    'min': (19/255, 54/255, 87/255),      # RGB (19, 54, 87)
    'dom': (24/255, 68/255, 110/255),     # RGB (24, 68, 110)
    'dim': (29/255, 82/255, 132/255),     # RGB (29, 82, 132)
    'aug': (33/255, 95/255, 154/255),     # RGB (33, 95, 154)
    '<pad>': 'gray'
}

def shift_pitch(pitch_str, shift):
    """
    Shifts the note pitch by the specified number of semitones.
    If the new pitch exceeds the maximum MIDI pitch, lowers it by one octave.
    Does not change '<pad>' and 'mtn'.
    
    Parameters:
    - pitch_str: str, the original pitch as a string
    - shift: int, number of semitones to shift
    
    Returns:
    - str, the shifted pitch
    """
    if pitch_str in ['<pad>', 'mtn']:
        return pitch_str
    try:
        pitch = int(pitch_str)
        new_pitch = pitch + shift
        while new_pitch > 127:
            new_pitch -= 12  # Lower by one octave
        while new_pitch < 0:
            new_pitch += 12  # Raise by one octave
        return str(new_pitch)
    except:
        return pitch_str

def shift_key(key_str, shift):
    """
    Shifts the root of the key by the specified number of semitones.
    Does not change '<pad>'.
    Example: 'C:maj' -> 'C#:maj' (shift=1)
    
    Parameters:
    - key_str: str, the original key string
    - shift: int, number of semitones to shift
    
    Returns:
    - str, the shifted key
    """
    if key_str == '<pad>':
        return key_str
    try:
        parts = key_str.split(':')
        if len(parts) != 2:
            return key_str  # Unexpected format
        root, chord_type = parts
        if chord_type not in CHORD_TYPES:
            return key_str  # Unexpected chord type
        if root not in NOTE_NAMES:
            return key_str  # Unexpected root note
        root_index = NOTE_NAMES.index(root)
        new_root_index = (root_index + shift) % 12
        new_root = NOTE_NAMES[new_root_index]
        return f"{new_root}:{chord_type}"
    except:
        return key_str

def shift_chord(chord_str, shift):
    """
    Shifts the root of the chord by the specified number of semitones.
    Does not change '<pad>'.
    Example: 'C:maj' -> 'C#:maj' (shift=1)
    
    Parameters:
    - chord_str: str, the original chord string
    - shift: int, number of semitones to shift
    
    Returns:
    - str, the shifted chord
    """
    return shift_key(chord_str, shift)

def load_data(x_path, y_path):
    """
    Loads the x_pop.npy and y_pop.npy files and returns them.
    
    Parameters:
    - x_path: str, path to the x_pop.npy file
    - y_path: str, path to the y_pop.npy file
    
    Returns:
    - tuple of np.ndarray: (x_pop, y_pop)
    """
    x_pop = np.load(x_path, allow_pickle=True)
    y_pop = np.load(y_path, allow_pickle=True)
    return x_pop, y_pop

def augment_data(x, y):
    """
    Combines x and y data, performs 12 semitone augmentations, and returns the augmented data.
    
    Parameters:
    - x: np.ndarray, shape (num_songs, num_steps, num_inputs=5)
    - y: np.ndarray, shape (num_songs, num_steps, num_outputs=2)
    
    Returns:
    - tuple of np.ndarray:
        - aug_x: shape (num_songs * 12, num_steps, num_inputs=5)
        - aug_y: shape (num_songs * 12, num_steps, num_outputs=2)
    """
    num_songs, num_steps, num_inputs = x.shape
    _, _, num_outputs = y.shape

    # Combine x and y into a single array
    combined = np.concatenate((x, y), axis=2)  # (num_songs, num_steps, 7)

    augmented_combined = []

    for shift in range(1, 13):  # Shifts from 1 semitone to 12 semitones
        shifted = combined.copy()
        for song in range(num_songs):
            for step in range(num_steps):
                # Shift note (index 0)
                note = shifted[song, step, 0]
                shifted[song, step, 0] = shift_pitch(note, shift)

                # Shift key (index 2)
                key = shifted[song, step, 2]
                shifted[song, step, 2] = shift_key(key, shift)

                # Shift chord (index 5)
                chord = shifted[song, step, 5]
                shifted[song, step, 5] = shift_chord(chord, shift)

        augmented_combined.append(shifted)

    # Concatenate all augmented data
    augmented_combined = np.concatenate(augmented_combined, axis=0)  # (num_songs * 12, num_steps, 7)

    # Split back into x and y
    aug_x = augmented_combined[:, :, :5]  # num_inputs=5 (note, bar, key, tempo, velocity)
    aug_y = augmented_combined[:, :, 5:]  # num_outputs=2 (chord, emotion)

    return aug_x, aug_y

def augment_single_song(x_sample, y_sample):
    """
    Performs 12 semitone augmentations on a single song and saves the augmented data.
    
    Parameters:
    - x_sample: np.ndarray, shape (num_steps, num_inputs=5)
    - y_sample: np.ndarray, shape (num_steps, num_outputs=2)
    
    Returns:
    - tuple of np.ndarray:
        - aug_x: shape (12, num_steps, num_inputs=5)
        - aug_y: shape (12, num_steps, num_outputs=2)
    """
    num_steps, num_inputs = x_sample.shape
    _, num_outputs = y_sample.shape
    
    aug_x = []
    aug_y = []
    
    for shift in range(1, 13):  # Shifts from 1 semitone to 12 semitones
        shifted_x = x_sample.copy()
        shifted_y = y_sample.copy()
        for step in range(num_steps):
            # Shift note
            note = shifted_x[step, 0]
            shifted_x[step, 0] = shift_pitch(note, shift)
            
            # Shift key
            key = shifted_x[step, 2]
            shifted_x[step, 2] = shift_key(key, shift)
            
            # Shift chord
            chord = shifted_y[step, 0]
            shifted_y[step, 0] = shift_chord(chord, shift)
        
        aug_x.append(shifted_x)
        aug_y.append(shifted_y)
    
    aug_x = np.array(aug_x)  # (12, num_steps, 5)
    aug_y = np.array(aug_y)  # (12, num_steps, 2)
    
    return aug_x, aug_y

def save_augmented_data(aug_x, aug_y, x_filename, y_filename, output_dir):
    """
    Saves the augmented data to the specified directory with filenames prefixed by 'aug_'.
    
    Parameters:
    - aug_x: np.ndarray, augmented x data
    - aug_y: np.ndarray, augmented y data
    - x_filename: str, original x file name
    - y_filename: str, original y file name
    - output_dir: str, directory to save the augmented data
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Directory '{output_dir}' created.")
    
    # Create augmented filenames
    aug_x_filename = f"aug_{x_filename}"
    aug_y_filename = f"aug_{y_filename}"
    
    # Save paths
    aug_x_path = os.path.join(output_dir, aug_x_filename)
    aug_y_path = os.path.join(output_dir, aug_y_filename)
    
    # Save the augmented data
    np.save(aug_x_path, aug_x)
    np.save(aug_y_path, aug_y)
    print(f"Augmented data saved to '{aug_x_path}' and '{aug_y_path}'.")

def main(x_path, y_path):
    """
    Loads a specific dataset, generates augmented data, and saves it.
    
    Parameters:
    - x_path: str, path to the x_pop.npy file
    - y_path: str, path to the y_pop.npy file
    """
    # Load data
    print(f"Loading data from '{x_path}' and '{y_path}'...")
    x_pop, y_pop = load_data(x_path, y_path)
    print(f"Original x shape: {x_pop.shape}")
    print(f"Original y shape: {y_pop.shape}")

    # Augment data
    print("Augmenting data...")
    aug_x, aug_y = augment_data(x_pop, y_pop)
    print(f"Augmented x shape: {aug_x.shape}")
    print(f"Augmented y shape: {aug_y.shape}")

    # Save augmented data
    x_filename = os.path.basename(x_path)
    y_filename = os.path.basename(y_path)
    output_dir = os.path.join(os.path.dirname(x_path), 'augmented')
    save_augmented_data(aug_x, aug_y, x_filename, y_filename, output_dir)

def augment_and_save_sample():
    """
    Performs 12 semitone augmentations on the first song and saves the augmented data to files.
    """
    # Set file paths
    x_pop_path = r'C:\Users\gauoo\jupyter_notebook\EMOPIA POP909 np CTC True\x_pop.npy'
    y_pop_path = r'C:\Users\gauoo\jupyter_notebook\EMOPIA POP909 np CTC True\y_pop.npy'
    
    # Load data
    print(f"Loading data from '{x_pop_path}' and '{y_pop_path}'...")
    x_pop, y_pop = load_data(x_pop_path, y_pop_path)
    print(f"Original x shape: {x_pop.shape}")
    print(f"Original y shape: {y_pop.shape}")
    
    # Extract the first song
    x_sample = x_pop[0, :, :]  # (num_steps, 5)
    y_sample = y_pop[0, :, :]  # (num_steps, 2)
    
    print("Original Data Sample:")
    print(f"First 10 Notes: {x_sample[:10, 0]}")
    print(f"First 10 Keys: {x_sample[:10, 2]}")
    print(f"First 10 Chords: {y_sample[:10, 0]}")
    
    # Augment data
    print("Augmenting data for the first song...")
    aug_x, aug_y = augment_single_song(x_sample, y_sample)
    print(f"Augmented x shape: {aug_x.shape}")
    print(f"Augmented y shape: {aug_y.shape}")
    
    # Save augmented data
    sample_output_dir = r'C:\Users\gauoo\jupyter_notebook\EMOPIA POP909 np CTC True\sample'
    if not os.path.exists(sample_output_dir):
        os.makedirs(sample_output_dir)
        print(f"Directory '{sample_output_dir}' created.")
    
    np.save(os.path.join(sample_output_dir, 'sample_aug_x.npy'), aug_x)
    np.save(os.path.join(sample_output_dir, 'sample_aug_y.npy'), aug_y)
    print("Augmented sample data saved to 'sample_aug_x.npy' and 'sample_aug_y.npy'.")

def verify_augmentation(original_x_path, original_y_path, augmented_x_path, augmented_y_path):
    """
    Compares parts of the original and augmented data to verify that augmentation was performed correctly.
    """
    original_x = np.load(original_x_path, allow_pickle=True)
    original_y = np.load(original_y_path, allow_pickle=True)

    augmented_x = np.load(augmented_x_path, allow_pickle=True)
    augmented_y = np.load(augmented_y_path, allow_pickle=True)

    # Verify augmented sample data
    print("\nAugmented Data Samples:")
    for i in range(12):
        print(f"\nShift={i+1}:")
        print(f"Notes: {augmented_x[i, :10, 0]}")
        print(f"Keys: {augmented_x[i, :10, 2]}")
        print(f"Chords: {augmented_y[i, :10, 0]}")



In [ ]:
if __name__ == '__main__':
    # Augment and save the entire dataset
    x_pop_path = r'example path ... x_pop.npy' # modify it properly
    y_pop_path = r'example path ... y_pop.npy'
    
    # Augment and save the entire dataset
    main(x_pop_path, y_pop_path)
    
    # Augment and save a sample song
    augment_and_save_sample()
    
    # Verify the augmented sample data
    sample_aug_x_path = r'example path ... sample_aug_x.npy'
    sample_aug_y_path = r'example path ... sample_aug_y.npy'
    
    print("\nVerifying augmented sample data...")
    verify_augmentation(
        x_pop_path,
        y_pop_path,
        sample_aug_x_path,
        sample_aug_y_path
    )
    
    print("\nAll datasets have been augmented, saved, and analyzed successfully.")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from fractions import Fraction

# Note name mapping (excluding octaves)
NOTE_NAMES = ['C', 'C#', 'D', 'D#', 'E', 'F', 
              'F#', 'G', 'G#', 'A', 'A#', 'B']

# Chord type mapping and color assignment (normalized to the 0-1 range for RGB)
CHORD_TYPES = ['maj', 'min', 'dom', 'dim', 'aug']
CHORD_TYPE_COLORS = {
    'maj': (14/255, 40/255, 65/255),      # RGB (14, 40, 65)
    'min': (19/255, 54/255, 87/255),      # RGB (19, 54, 87)
    'dom': (24/255, 68/255, 110/255),     # RGB (24, 68, 110)
    'dim': (29/255, 82/255, 132/255),     # RGB (29, 82, 132)
    'aug': (33/255, 95/255, 154/255),     # RGB (33, 95, 154)
    '<pad>': 'gray'
}

def pitch_to_note(pitch):
    """
    Converts a MIDI pitch to a note name (excluding octaves).
    Returns '<pad>' unchanged.
    
    Parameters:
    - pitch: str or int, the original pitch
    
    Returns:
    - str, the corresponding note name or '<pad>'
    """
    if isinstance(pitch, str) and pitch == '<pad>':
        return '<pad>'
    try:
        pitch = int(pitch)
        note_index = pitch % 12
        return NOTE_NAMES[note_index]
    except:
        return '<pad>'

def extract_chord_type(chord):
    """
    Extracts the chord type from a chord string (e.g., 'C:maj' -> 'maj').
    Returns '<pad>' unchanged.
    
    Parameters:
    - chord: str, the original chord string
    
    Returns:
    - str, the chord type or '<pad>'
    """
    if chord == '<pad>':
        return '<pad>'
    try:
        parts = chord.split(':')
        if len(parts) != 2:
            return '<pad>'
        chord_type = parts[1]
        if chord_type not in CHORD_TYPES:
            return '<pad>'
        return chord_type
    except:
        return '<pad>'

def load_data(x_path, y_path):
    """
    Loads the x_pop.npy and y_pop.npy files and returns them.
    
    Parameters:
    - x_path: str, path to the x_pop.npy file
    - y_path: str, path to the y_pop.npy file
    
    Returns:
    - tuple of np.ndarray: (x_pop, y_pop)
    """
    x_pop = np.load(x_path, allow_pickle=True)  # Adjust allow_pickle as needed
    y_pop = np.load(y_path, allow_pickle=True)
    return x_pop, y_pop

def process_notes(x_pop, num_steps):
    """
    Processes the note features to return a list of note names.
    If a note occurs, it is filled from its position until before the next note occurs.
    When calculating cumulative statistics across the entire dataset, all songs and time steps are included.
    
    Parameters:
    - x_pop: np.ndarray, input features (num_songs, num_steps, num_inputs=5)
    - num_steps: int, number of time steps
    
    Returns:
    - list of str: melody sequence with notes filled appropriately
    """
    # 'note' feature is x_pop[:, :, 0]
    notes = x_pop[:, :, 0].flatten()
    melody_sequence = []
    
    current_note = '<pad>'
    for note in notes:
        if note != '<pad>':
            current_note = pitch_to_note(note)
        melody_sequence.append(current_note)
    
    # No need to fill beyond the actual song length as the data is already flattened
    return melody_sequence

def process_chords(y_pop, num_steps):
    """
    Processes the chord features to return a list of chords.
    When calculating cumulative statistics across the entire dataset, all songs and time steps are included.
    
    Parameters:
    - y_pop: np.ndarray, output features (num_songs, num_steps, num_outputs=2)
    - num_steps: int, number of time steps
    
    Returns:
    - list of str: chord sequence with chords filled appropriately
    """
    # 'chord' feature is y_pop[:, :, 0]
    chords = y_pop[:, :, 0].flatten()
    chord_sequence = []
    
    current_chord = '<pad>'
    for chord in chords:
        if chord != '<pad>':
            current_chord = chord
        chord_sequence.append(current_chord)
    
    # No need to fill beyond the actual song length as the data is already flattened
    return chord_sequence

def process_chord_types(chord_sequence):
    """
    Processes the chord sequence to return a list of chord types.
    
    Parameters:
    - chord_sequence: list of str, the chord sequence
    
    Returns:
    - list of str: chord type sequence
    """
    chord_type_sequence = [extract_chord_type(chord) for chord in chord_sequence]
    return chord_type_sequence

def plot_histograms(note_sequence, chord_sequence, chord_type_sequence, dataset_name):
    """
    Plots histograms for notes, chords, and chord types. Adds black edges to each bar.
    
    Parameters:
    - note_sequence: list of str, the note sequence
    - chord_sequence: list of str, the chord sequence
    - chord_type_sequence: list of str, the chord type sequence
    - dataset_name: str, name of the dataset (used in histogram titles)
    """
    # Note histogram
    note_counter = Counter(note_sequence)
    note_classes = NOTE_NAMES + ['<pad>']
    note_freq = [note_counter.get(cls, 0) for cls in note_classes]
    
    # Chord histogram
    chord_counter = Counter(chord_sequence)
    # 12 roots * 5 types + <pad> = 61 classes
    roots = NOTE_NAMES
    chord_types = CHORD_TYPES
    chord_classes = [f"{root}:{ctype}" for root in roots for ctype in chord_types] + ['<pad>']
    chord_freq = [chord_counter.get(cls, 0) for cls in chord_classes]
    chord_colors = []
    for cls in chord_classes:
        if cls == '<pad>':
            chord_colors.append(CHORD_TYPE_COLORS['<pad>'])
        else:
            ctype = extract_chord_type(cls)
            chord_colors.append(CHORD_TYPE_COLORS.get(ctype, 'gray'))
    
    # Chord type histogram
    chord_type_counter = Counter(chord_type_sequence)
    chord_type_classes = CHORD_TYPES
    chord_type_freq = [chord_type_counter.get(cls, 0) for cls in chord_type_classes]
    chord_type_colors = [CHORD_TYPE_COLORS[cls] for cls in chord_type_classes]
    
    # Set up subplots
    fig, axs = plt.subplots(3, 1, figsize=(12, 18))  # Width:Height ratio approximately 2:1
    
    # 1. Note histogram
    axs[0].bar(note_classes, note_freq, color='gray', edgecolor='black')  # Added edgecolor
    axs[0].set_title(f'Note Frequency - {dataset_name}')
    axs[0].set_xlabel('Note')
    axs[0].set_ylabel('Frequency')
    axs[0].set_xticks(range(len(note_classes)))
    axs[0].set_xticklabels(note_classes, rotation=45)
    
    # 2. Chord histogram
    axs[1].bar(chord_classes, chord_freq, color=chord_colors, edgecolor='black')  # Added edgecolor
    axs[1].set_title(f'Chord Frequency - {dataset_name}')
    axs[1].set_xlabel('Chord')
    axs[1].set_ylabel('Frequency')
    axs[1].set_xticks(range(len(chord_classes)))
    axs[1].set_xticklabels(chord_classes, rotation=90)
    
    # 3. Chord type histogram
    axs[2].bar(chord_type_classes, chord_type_freq, color=chord_type_colors, edgecolor='black')  # Added edgecolor
    axs[2].set_title(f'Chord Types Frequency - {dataset_name}')
    axs[2].set_xlabel('Chord Type')
    axs[2].set_ylabel('Frequency')
    axs[2].set_xticks(range(len(chord_type_classes)))
    axs[2].set_xticklabels(chord_type_classes, rotation=45)
    
    plt.tight_layout()
    plt.savefig(f'{dataset_name}.png')
    plt.show()

def main(x_path, y_path, dataset_name):
    """
    Loads a specific dataset and visualizes statistics for notes and chords.
    
    Parameters:
    - x_path: str, path to the x_pop.npy file
    - y_path: str, path to the y_pop.npy file
    - dataset_name: str, name of the dataset (used in histogram titles)
    """
    # Load data
    x_pop, y_pop = load_data(x_path, y_path)
    
    # Check data shape (e.g., (num_songs, num_steps, num_features))
    num_samples, num_steps, num_input = x_pop.shape
    _, _, num_output = y_pop.shape
    print(f"Dataset: {dataset_name}")
    print(f"x_pop shape: {x_pop.shape}")
    print(f"y_pop shape: {y_pop.shape}")
    
    # Process note and chord sequences
    note_sequence = process_notes(x_pop, num_steps)
    chord_sequence = process_chords(y_pop, num_steps)
    chord_type_sequence = process_chord_types(chord_sequence)
    
    # Plot histograms
    plot_histograms(note_sequence, chord_sequence, chord_type_sequence, dataset_name)



In [ ]:
if __name__ == '__main__':
    # Set file paths
    x_pop_path = r'example path ... x_pop.npy'
    y_pop_path = r'example path ... y_pop.npy'

    aug_x_pop_path = r'example path ... aug_x_pop.npy'
    aug_y_pop_path = r'example path ... aug_y_pop.npy'
    
    # Process datasets and plot histograms
    main(x_pop_path, y_pop_path, 'POP909')
    main(aug_x_pop_path, aug_y_pop_path, 'Augmented POP909')

    print("All datasets have been processed and histograms generated successfully.")
